<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/nophishing/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No Phishing: Detecting Malicious URLs
## 02: Models and Evaluation

## Data Preparation

The dataset consistes of the raw URL, the binary phishing label, and 87 features, including:

* 56 from URL syntax and structure,
* 24 from page content, and
* 7 from external services.

Little to no cleaning is necessary.  However, some variables do include apparently-invalid values. TODO add results from EDA

I dropped the raw URL, as the modeling techniques can only make effective use of quantitative features.







## Data Preparation

Prior to beginning the preprocessing pipeline, the following transformations are made:
* Transform the target variable **status** to binary, where 1 = 'phishing' and 0 = 'legitimate', and rename the column to **y**
* Drop the raw URL
* TODO - following EDA, consider dropping rows with bad data
* Split the **training** dataset into X and y divisions (separating the features from the target variable) and each of these into training and validation sets.  Note: The **test** dataset will not be loaded until final model evaluation.


In [ ]:
# Load and prepare training data
import pandas as pd

train_url = 'https://raw.githubusercontent.com/neal-logan/dsba6211-summer2024/main/nophishing/data/phishing-url-pirochet-train.csv'
df = pd.read_csv(train_url)

#Create numeric target variable column
df['y'] = df['status'].replace('legitimate', 0).replace('phishing', 1)

#Drop unnecessary columns
df = df.drop(columns=['status','url'])

#X/y split
X = df.drop(columns=['y'])
y = df['y']

In [ ]:
#Split training set into training and validation set (test set not yet loaded)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42)

#Recombine X,y training data for exploration
Xy_train = X_train.copy()
Xy_train['y'] = y_train


## Preprocessing Pipeline


https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#plot-all-scaling-standard-scaler-section

Probably use mostly robust scaling




In [ ]:
# TODO Develop the preprocessing pipeline for selected features

#### Baseline Model Performance

Note that performance of RF and GBT on the validation set was strong enough that further development may not make sense from the perspective of precision/recall/AUC metrics.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
import pandas as pd

# Define model evaluation function

def print_model_evaluation(
    title: str,
    pipe : Pipeline,
    X : pd.DataFrame,
    y : pd.DataFrame):

    print("\n" + title)
    pred_y = pipe.predict(X)
    print(confusion_matrix(pred_y, y))
    print("\nROC-AUC: " + str(roc_auc_score(pred_y, y)))
    print("Precision: " + str(precision_score(pred_y, y)))
    print("Recall: " + str(recall_score(pred_y, y)))


#### Logistic Regression

In [ ]:
#Set up pipeline

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(
      StandardScaler(),
      LogisticRegression(random_state=42)
)

pipe_lr.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(random_state=42))])

In [ ]:
print_model_evaluation("Logistic Regression\nPerformance on Training Set",
                       pipe_lr, X_train, y_train)

print_model_evaluation("Logistic Regression\nPerformance on Validation Set",
                       pipe_lr, X_validation, y_validation)

Logistic Regression
Performance on Training Set
[[2934  158]
 [ 142 2892]]

Area under ROC curve: 0.9510487438184406

Precision: 0.9481967213114754

Recall: 0.9531970995385629
Logistic Regression
Performance on Validation Set
[[711  44]
 [ 42 735]]

Area under ROC curve: 0.9438339001252909

Precision: 0.9435173299101413

Recall: 0.9459459459459459


#### Gradient-boosted Trees

In [ ]:
# Set up and run pipeline - gradient boosted trees

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline

pipe_gbt = make_pipeline(
      StandardScaler(),
      HistGradientBoostingClassifier(random_state=42)
)

pipe_gbt.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(random_state=42))])

In [ ]:
print_model_evaluation("Gradient-boosted Trees\nPerformance on Training Set",
                       pipe_gbt, X_train, y_train)

print_model_evaluation("Gradient-boosted Trees\nPerformance on Validation Set",
                       pipe_gbt, X_validation, y_validation)

Gradient-boosted Trees
Performance on Training Set
[[3076    0]
 [   0 3050]]

Area under ROC curve: 1.0

Precision: 1.0

Recall: 1.0
Gradient-boosted Trees
Performance on Validation Set
[[728  26]
 [ 25 753]]

Area under ROC curve: 0.9666917826433827

Precision: 0.9666238767650834

Recall: 0.967866323907455
